## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Изучение общей информации

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
from pymystem3 import Mystem


In [2]:
data = pd.read_csv('data.csv')

Выведем первые 10 строк таблицы, а также общую информацию.

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div style="border:solid green 2px; padding: 20px">
В таблице 12 столбцов, имебщих типы *int*, *float* и *object*. Количество записей в столбцах неодинаково - в данных есть пропуски, от которых нужно избавиться.

### Предобработка данных


#### Обработка пропусков

Найдем количество пропусков в каждом из столбцов таблицы.

In [5]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Выведем первые 5 строк, в которых имеются пропуски в столбце стажа.

In [6]:
data[data['days_employed'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Пропуски в столбце "стаж работы" совпадают с пропусками в столбце "доход". Проверим, во всех ли это строках.

In [7]:
data[(data['days_employed'].isnull()) & (data['total_income'].isnull())].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Получили 2174 строки, в которых существуют пропуски в обоих столбцах. Именно столько строк было с пропусками в каждом из этих столбцов, следовательно, появление этих пропусков как-то связано.

Заполним медианой пропущенные значения в столбе дохода.

In [8]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

Данные столбца стажа не пригодятся для ответа на ключевые вопросы вопросы исследования. К тому же эти данные содержат огромное число ошибок (отрицательные или неправдоподобно большие значения). Приведение данных к виду, в котором они будут пригодны для анализа (в частности для заполнения пропущенных значений медианой), займет много времени. Но удалять строки с пропущенными значениями нельзя, так как потеряется часть важных для нас данных, так что сочтем целесообразным заполнить пропущенные значения в столбце трудового стажа нулем.

In [9]:
data['days_employed'] = data['days_employed'].fillna(0)

Проверим, не осталось ли в данных больше пропусков.

In [10]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все значения нули, значит пропуски в данных мы устранили.

<div style="border:solid green 2px; padding: 20px">
На данном этапе мы разобрались с пропусками в данных и устранили их (в одном случае заполнив пропуски медианой, в другом - нулем).
Также мы выяснили, что появление пропусков в столбцах "стаж работы" и "доход" связано. Это можно объяснить, например, тем, что человек никогда не работал, следовательно стажа работы не имел и зарплату не получал. Либо тем, что человек не указал данные по своей работе. Либо его оформление на работу было неправильно произведено, стаж не начал считаться и заработная плата не отображается. Либо во всем виноват технический сбой :)

#### Замена типа данных

При просмотре общей информации таблицы были отмечены два столба, имеющие тип данных **float**. Мы уже решили, что не будем модифицировать столбец стажа работы, однако со столбцом доход работать мы будем. 

Значительное количество знаков после запятой затрудняет работу с числами. Но не помешает ли анализу округление чисел? Не исказятся ли нужные данные? Найдем средние значения по столбцу до и после приведения типа и выведем их на экран.

In [11]:
total_income_median = data['total_income'].mean()
total_income_median_after = data['total_income'].astype('int').mean()
print(total_income_median, total_income_median_after)

165159.4873972547 165158.94276422763


Значение среднего изменилось всего на единицу, значит можем безбоязненно заменить тип **float** на **int**.

In [12]:
#применим метод astype, позволяющий перевести данные в нужный тип
data['total_income'] = data['total_income'].astype('int')
#выведем столбец на экран, чтобы  удостовериться в произведенных измененениях
print(data['total_income'])

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int32


<div style="border:solid green 2px; padding: 20px">
Мы изменили тип данных с неудобного для быстрой оценки на удобный, приятный глазу. Это ли не счастье!

#### Обработка дубликатов

При просмотре общей информации таблицы были замечены записи в столбце "образование", сделанные в верхнем регистре. Проверим, так ли это.

In [13]:
print(data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


Всего видов образования 5, а здесь вариантов гораааздо больше. Исправим это, приведя все записи к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Вот так гораздо лучше.

Подобная проблема была замечена и в столбце "семейное положение". Исправим это.

In [15]:
data['family_status'] = data['family_status'].str.lower()

---

При просмотре общей информации таблицы были замечены странные значения в стобце "дети". Выведем все уникальные значения этого столбца.

In [16]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


Отрицательные дети - явление интересное, но для анализа не пригодное. Посмотрим информацию о строках  такими данными.

In [17]:
print(data[data['children'] == -1]['children'].count())
print(data[data['children'] == -1].head())

47
     children  days_employed  dob_years education  education_id  \
291        -1   -4417.703588         46   среднее             1   
705        -1    -902.084528         50   среднее             1   
742        -1   -3174.456205         57   среднее             1   
800        -1  349987.852217         54   среднее             1   
941        -1       0.000000         57   среднее             1   

             family_status  family_status_id gender  income_type  debt  \
291       гражданский брак                 1      F    сотрудник     0   
705        женат / замужем                 0      F  госслужащий     0   
742        женат / замужем                 0      F    сотрудник     0   
800  не женат / не замужем                 4      F    пенсионер     0   
941        женат / замужем                 0      F    пенсионер     0   

     total_income                       purpose  
291        102816        профильное образование  
705        137882       приобретение автомобиля  

Возраст у всех людей разный, семейное положение тоже, поэтому нельзя однозначно определить реальное количество детей, которое должно было быть на месте ошибки. Следовательно, эти строки нам для анализа не помогут. Так как их количество относительно общего мало, можно их удалить, не потеряв важные данные.

Проделаем то же самое с детьми в количестве 20 человек.

In [18]:
print(data[data['children'] == 20]['children'].count())
print(data[data['children'] == 20].head())

76
      children  days_employed  dob_years education  education_id  \
606         20    -880.221113         21   среднее             1   
720         20    -855.595512         44   среднее             1   
1074        20   -3310.411598         56   среднее             1   
2510        20   -2714.161249         59    высшее             0   
2941        20   -2161.591519          0   среднее             1   

        family_status  family_status_id gender income_type  debt  \
606   женат / замужем                 0      M   компаньон     0   
720   женат / замужем                 0      F   компаньон     0   
1074  женат / замужем                 0      F   сотрудник     1   
2510   вдовец / вдова                 2      F   сотрудник     0   
2941  женат / замужем                 0      F   сотрудник     0   

      total_income                                purpose  
606         145334                          покупка жилья  
720         112998                   покупка недвижимости  

Аналогичный вывод: удаляем.

In [19]:
data['children'] = data[data['children'] != 20]
data['children'] = data[data['children'] != -1]
#найдем количество пропусков, затем удалим их и проверим, остались ли они еще
print(data['children'].isna().sum())
data = data.dropna()

123


In [20]:
print(data['children'].isna().sum())

0


---

Посмотрим, есть ли у нас полные дубликаты. Если да, удалим их, а затем проверим, остались ли они в таблице.

In [21]:
print(data.duplicated().sum())

71


In [22]:
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())

0


<div style="border:solid green 2px; padding: 20px">
На данном шаге нам удалось найти дубликаты (с учетом регистра и полные) и удалить их. Также отыскались неправдоподобные значения в столбце "дети", с которыми мы решили не работать, так как они не сильно повлияют на итоговые результаты исследования, но помешают анализу. 
Почему они могли возникнуть? Подразумеваем, что виноваты технические сбои.

### Лемматизация

In [23]:
m = Mystem()

Будем лемматизировать столбец, содержащий цели взятых кредитов. Для этого создадим функцию, а затем с помощью метода *apply* применим ее к требуемому столбцу. Полученные значения запишем в новый столбец purpose_lemmatise. Затем найдем леммы, используемые чаще всего. Для этого "сложим" значения нового столбца в одну строку и применим к ней метод *Counter()* (складываем все потому, что counter может работать только с одной строкой).

In [24]:
def purposes_lemm (purpose):
    lemma = m.lemmatize(purpose)
    return lemma
data['purpose_lemmatise'] = data['purpose'].apply(purposes_lemm)

purpose_sum = data['purpose_lemmatise'].sum()
from collections import Counter
print(Counter(purpose_sum))

KeyboardInterrupt: 

Наиболее частые цели кредита - на недвижимость и жилье (объединим их в одну категорию), автомобиль, образование, свадьбу. Создадим новый столбец - финальную лемматизацию, в котором промаркируем цели кредитов обозначенными частыми категориями.

In [ ]:
def purposes_lemm_final (purpose):
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    
data['purpose_lemmatise_final'] = data['purpose_lemmatise'].apply(purposes_lemm_final)

#проверим, все ли работает так, как нам нужно
print(data['purpose_lemmatise_final'].head())

<div style="border:solid green 2px; padding: 20px">
На данном шаге мы лемматизировали столбец, содержащий цели кредитов. Нашли ключевые слова в целях и промаркировали каждую запись ключевым словом. Оказалось, что целей на деле гораздо меньше, чем могло показаться изначально! Это облегчит работу с данными, а также поможет ответить на основные вопросы исследования.

### Категоризация данных

Для начала создадим таблицу, включающую полезные для анализа данные (исключим бесполезный для нас столбец стажа работы). Также не будем включать столбцы, содержащие данные по образованию и семейному положению - оставим лишь их идентификаторы, а словари с этими данными соберем в других таблицах.

In [ ]:
new_data = data[['children', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose_lemmatise_final']]

education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop = True)
print(education_dict)

family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop = True)
print(family_status_dict)

Сгруппируем таблицу так, чтобы было удобнее отвечать на основные вопросы исследования.
1.  **по количеству детей** (0 - нет детей, 1-2 - один/два ребенка, >3 - многодетные)
2.  **по семейному положению** (женат/замужем + гражданский брак - в браке, не женат/не замужем - не был в браке никогда, другое - сейчас не в браке)
3.  **по уровню дохода** (разобьем на группы: низкий, средний, высокий)

In [ ]:
#1
def child_func (children):
    if children == 0:
        return 'нет детей'
    elif children == 1 or children == 2:
        return 'один/два ребенка'
    else: 
        return 'многодетные'

data['children_type'] = data['children'].apply(child_func)
print(data['children_type'].head())

In [ ]:
#2
def fam_func (family):
    if family == 0 or family == 1:
        return 'в браке'
    elif family == 4:
        return 'не был в браке никогда'
    else:
        return 'сейчас не в браке'
    
data['family_status_type'] = data['family_status_id'].apply(fam_func)
print(data['family_status_type'].head())

---

Как поделить клиентов по уровню дохода? Для начала найдем максимальный, средний и минимальный (отличный от нуля) доход.

In [ ]:
print('Максимальный доход =', data['total_income'].max())
print('Средний доход (медиана) =', int(data['total_income'].median()))
print('Минимальный доход =', data[data['total_income'] != 0]['total_income'].min())

Наблюдаем сильный разброс. Вероятно, в таблице есть артефакты - люди с огромным доходом. Найдем процент людей с доходом более 400 тысяч. 

In [ ]:
print('{:.2%}'.format(data[data['total_income'] > 300000]['total_income'].count() / len(data)))

7 процентов! Отнесем людей с таким доходом к олигархам, а остальных разделим на 3 группы: от 0 до 100 тысяч - низкий доход, от 100 до 200 тысяч - средний, > 200 тысяч - высокий.

In [ ]:
def total_income_func (income):
    if income <= 100000:
        return 'низкий'
    elif income <= 200000:
        return 'средний'
    elif income <= 300000:
        return 'высокий'
    else:
        return 'олигарх'
    
data['income_type'] = data['total_income'].apply(total_income_func)
print(data['income_type'].head())

<div style="border:solid green 2px; padding: 20px">
Мы категоризовали данные так, чтобы дальнейшем с ними было удобнее работать. В процессе категоризации были выявлены артефакты в столбце дохода - олигархи. Они были выделены в отдельный столбец.

### Промежуточные результаты и выводы

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Попробуем ответить на данный вопрос. 

In [ ]:
#создадим 2 переменные
#в одну положим сгруппированное по столбцу children_type количество детей 
#в другую - сгруппированную по тому же столбцу сумму задолженностей:
#помним, что имеющие долг имеют идентификатор 1 в столбце debt, поэтому просто просуммируем их
child_type_number = data.groupby('children_type')['children'].count()
child_type_debt_number = data.groupby('children_type')['debt'].sum()

#найдем отношение количества людей в каждой категории с долгом к общему количеству в категории
child_debt_percent = child_type_debt_number / child_type_number

#выведем на экран в процентном соотношении
print((child_debt_percent * 100))

#прим: чем больше процент, тем хуже отдают кредиты

Бездетным получается выгоднее всего давать кредиты!

#### Вывод

Получилось, что люди, не имеющие детей, стабильнее отдают кредиты, нежели чем люди с детьми. Почему? Скорее всего потому, что без детей меньше постоянных расходов и непредвиденных трат, больше возможностей и способов для заработка. Больше свободы действий!

Также данные разнятся в зависимости от числа детей - мало/много. Многодетные семьи лучше отдают кредиты потому, что люди уже старше, имеют опыт в семейной экономике, громотно оценивают доходы/расходы семьи. Также у многодетных семей есть льготы от государства: мат. капитал, льготы на транспорт, на жилье, выплаты. Семьи используют эти деньги как дополнительный источник дохода.

Люди, имеющие одного/двух детей, выплачивают кредиты хуже всех, потому что такие семьи обычно молодые, не до конца осознающие семейный бюджет и риски невыплат. Льготы от государства незначительные и не покрывают реальные расходы.

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Алгоритм действий здесь схож с алгоритмом предыдущего шага.

In [ ]:
family_type_number = data.groupby('family_status_type')['children'].count()

family_type_debt_number = data.groupby('family_status_type')['debt'].sum()

family_debt_percent = family_type_debt_number / family_type_number

print((family_debt_percent * 100))

Не был в браке - кредит не получишь!

#### Вывод

Велик процент невозврата кредита среди тех, кто никогда не был в браке. Это можно объяснить тем, что это молодые люди, еще не совсем ответственно относящиеся к деньгам, плохо управляющие собственными финансами, возможно даже безруссудно. Рассчитывать они могут только на себя или родственников (родителей, друзей).

Далее идут те, кто находится в браке. Они понимают ценность денег, несут ответственность за семью. Но могут иметь непредвиденные расходы - супруга внезапно захотела шубу, как тут откажешь. И кредит отходит на второй план.

И наконец - находящиеся в разводе или вдовы/вдовцы. Такие люди отдают кредиты лучше всего, потому что они уже познали семейную жизнь, научились следить за деньгами, но в данный момент отвечают только за свои расходы и полагаться могут только на себя. А кому можно больше всего доверять, как не себе.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [ ]:
income_type_number = data.groupby('income_type')['children'].count()

income_type_debt_number = data.groupby('income_type')['debt'].sum()

income_debt_percent = income_type_debt_number / income_type_number

print((income_debt_percent * 100))

Получили довольно неожиданный результат - люди со средним доходом (100-200 тысяч) хуже всех отдают кредиты. А безопаснее всего занимать людям с высоким доходом.

#### Вывод

Люди с высоким и аномально высоким доходом хорошо отдают кредиты просто потому, что им есть из чего отдавать. Таким давать деньги безопасно.

Люди с низким доходом отдают взятые деньги хуже, потому что надо сначала найти, откуда взять денег, чтобы отдать. Однако у людей с небольшим доходом вероятно более ответственное отношение к деньгам, есть строго расписанный бюджет и мотивация не просрочить выплату. 

Людям со средним доходом давать кредиты получается небезопасно. Вероятно, они менее стремятся к финансовому росту, ведь на бытовые нужды заработной платы хватает. К тому же банки довольно охотно дают кредиты людям со средним доходом, и кто-то мог набрать много и получить себе непосильную долговую нагрузку.

**- Как разные цели кредита влияют на его возврат в срок?**

In [ ]:
purpose_type_number = data.groupby('purpose_lemmatise_final')['children'].count()

purpose_type_debt_number = data.groupby('purpose_lemmatise_final')['debt'].sum()

purpose_debt_percent = purpose_type_debt_number / purpose_type_number

print(purpose_debt_percent * 100)

Получается, возврат кредитов, взятых на автомобили и образование, чаще всего задерживают! 

#### Вывод

Менее всего задерживают выплату кредитов на недвижимость, затем идут кредиты, взятые на свадьбу. Как это можно объяснить? В кредитах, берущихся на жилье, суммы больше всего, большой по размерам первоначальный взнос. Больше и проверок со стороны банка. 

Кредиты на свадьбы и автомобили - так называемые потребительские. Можно сравнить с кредитом на айфон: сумма гораздо меньше, нежели чем на недвижимость, проверок меньше. Но за свадьбу деньги можно отдать за счет важного источника дохода - денег, подаренных на свадьбу. А кредиты на автомобили обычно берут люди со средним или низким доходом, и им бывает сложно их выплачивать.

Деньги, взятые на образование, не отдают так же часто, как и на автомобили. Это можно объяснить тем, что кредиты берут студенты, которые не работают во время учебы - просто не успевают. Естественно, до некоторого времени отдавать деньги им бывает нечем. Также кредиты берут уже на дополнительное образование с целью повышения квалификации, а с ней и нахождения новой работы, роста зарплаты. Но кто-то либо не доучивается, либо не находит после работу - ожидаемых денег нет, кредит выплачивать нечем.

Получается, наименее рискованно для банка давать кредиты на недвижимость (но там и суммы больше). Наиболее рискованно - на автомобили и образование.

### Общий вывод

В ходе исследования получили портрет человека, который с наибольшей вероятностью вернет деньги банку: женатый, но без детей, с высоким доходом, берущий кредит на недвижимость. И, наоборот, кому не следует доверять деньги: имеющему малое количество детей, но не семейному, со средним доходом, желающему приобрести автомобиль (такой индивид вряд ли появится по объективным причинам).

 Однако это только промежуточные, сырые результаты. Для достоверной оценки необходимо объединить все полученные ответы на вопросы, добавить дополнительные данные (срок кредита, сумма), и тогда все выданные кредиты будут вовремя возвращаться к банкам!